In [1]:
# Test flyte tasks and workflows in a notebook

from wine_classification_hyperparameter_search_v3 import (
    get_data,
    split_data,
    create_search_grid,
    train_model,
    compare_model_results,
    training_workflow,
    analyze_model
)
from functools import partial
from flytekit import map_task

%load_ext autoreload
%autoreload 2

/Users/tchase/anaconda3/envs/union-poc-env/lib/python3.11/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


00:01:27.908170 WARNING  type_engine.py:1358 - Unsupported Type typing.Any      
                         found, Flyte will default to use PickleFile as the     
                         transport. Pickle can only be used to send objects     
                         between the exact same version of Python, and we       
                         strongly recommend to use python type that flyte       
                         support.

## Test Flyte Tasks

In [2]:
%env

{'COMMAND_MODE': 'unix2003',
 'HOME': '/Users/tchase',
 'LOGNAME': 'tchase',
 'LaunchInstanceID': '4900FAF6-3273-449F-B911-C431ACDF7FD2',
 'MallocNanoZone': '0',
 'ORIGINAL_XDG_CURRENT_DESKTOP': 'undefined',
 'PATH': '/Users/tchase/anaconda3/envs/union-poc-env/bin:/Users/tchase/anaconda3/condabin:/Users/tchase/.rd/bin:/usr/bin:/bin:/usr/sbin:/sbin:/usr/local/bin',
 'SECURITYSESSIONID': '186b3',
 'SHELL': '/bin/zsh',
 'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.SzMZdijyOx/Listeners',
 'TMPDIR': '/var/folders/_9/60k4g7zj1wq1kyl_pr7hq3k80000gq/T/',
 'USER': 'tchase',
 'VSCODE_AMD_ENTRYPOINT': 'vs/workbench/api/node/extensionHostProcess',
 'VSCODE_CODE_CACHE_PATH': '/Users/tchase/Library/Application Support/Code/CachedData/f1e16e1e6214d7c44d078b1f0607b2388f29d729',
 'VSCODE_CRASH_REPORTER_PROCESS_TYPE': 'extensionHost',
 'VSCODE_CWD': '/',
 'VSCODE_HANDLES_UNCAUGHT_ERRORS': 'true',
 'VSCODE_IPC_HOOK': '/Users/tchase/Library/Application Support/Code/1.91-main.sock',
 'VSCODE_NLS_CONFIG

In [3]:
# Define hyperparameter grid for tasks

grid = {
    'max_depth': [10, 50, 100],
    'max_features': [None, 'sqrt'],
    'n_estimators': [100, 2000]
}

In [4]:
%load_ext dotenv
%dotenv

# Test get_data task

data = get_data()
data.head()

WANDB_API_KEY


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /Users/tchase/Documents/repos/flyte_union_public/wine_classification_hyperparameter_search_v3.py │
│ :45 in get_data                                                                                  │
│                                                                                                  │
│    42 # @task(container_image=sklearn_image_spec)                                                │
│    43 def get_data() -> pd.DataFrame:                                                            │
│    44 │   print('WANDB_API_KEY')                                                                 │
│ ❱  45 │   print(current_context().secrets.get(key="WANDB_API_KEY"))                              │
│    46 │   # print('FBIN_SNOWFLAKE_USER')                                                         │
│    47 │   # print(current_context().secrets.get(key="FBIN_SNOWFLAKE_USER"))                      │
│    48 │   # Load Data                                                                            │
│                                                                                                  │
│ /Users/tchase/anaconda3/envs/union-poc-env/lib/python3.11/site-packages/flytekit/core/context_ma │
│ nager.py:384 in get                                                                              │
│                                                                                                  │
│   381 │   │   if os.path.exists(fpath):                                                          │
│   382 │   │   │   with open(fpath, encode_mode) as f:                                            │
│   383 │   │   │   │   return f.read().strip()                                                    │
│ ❱ 384 │   │   raise ValueError(                                                                  │
│   385 │   │   │   f"Please make sure to add secret_requests=[Secret(group={group}, key={key})]   │
│   386 │   │   │   f"in Env Var:{env_var} and FilePath: {fpath}"                                  │
│   387 │   │   )                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Please make sure to add secret_requests=[Secret(group=None, key=WANDB_API_KEY)] in @task. Unable to 
find secret for key WANDB_API_KEY in group None in Env Var:_FSEC_WANDB_API_KEY and FilePath: 
/etc/secrets/wandb_api_key

╭─────────────────────────────────────── Traceback (most recent call last) ───────────────────────────────────────╮
│ /Users/tchase/anaconda3/envs/union-poc-env/lib/python3.11/site-packages/flytekit/exceptions/scopes.py:217 in    │
│ user_entry_point                                                                                                │
│                                                                                                                 │
│ ❱ 217 │   │   │   │   return wrapped(*args, **kwargs)                                                           │
│                                                                                                                 │
│ /Users/tchase/Documents/repos/flyte_union_public/wine_classification_hyperparameter_search_v3.py:45 in get_data │
│                                                                                                                 │
│ ❱  45 │   print(current_context().secrets.get(key="WANDB_API_KEY"))                                             │
│                                                                                                                 │
│ /Users/tchase/anaconda3/envs/union-poc-env/lib/python3.11/site-packages/flytekit/core/context_manager.py:384 in │
│ get                                                                                                             │
│                                                                                                                 │
│ ❱ 384 │   │   raise ValueError(                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Please make sure to add secret_requests=[Secret(group=None, key=WANDB_API_KEY)] in @task. Unable to 
find secret for key WANDB_API_KEY in group None in Env Var:_FSEC_WANDB_API_KEY and FilePath: 
/etc/secrets/wandb_api_key

During handling of the above exception, another exception occurred:

╭─────────────────────────────────────── Traceback (most recent call last) ───────────────────────────────────────╮
│ in <module>:6                                                                                                   │
│                                                                                                                 │
│ ❱ 6 data = get_data()                                                                                           │
│                                                                                                                 │
│ /Users/tchase/anaconda3/envs/union-poc-env/lib/python3.11/site-packages/flytekit/core/base_task.py:358 in       │
│ __call__                                                                                                        │
│                                                                                                                 │
│ ❱ 358 │   │   return flyte_entity_call_handler(self, *args, **kwargs)  # type: ignore                           │
│                                                                                                                 │
│ /Users/tchase/anaconda3/envs/union-poc-env/lib/python3.11/site-packages/flytekit/core/promise.py:1261 in        │
│ flyte_entity_call_handler                                                                                       │
│                                                                                                                 │
│ ❱ 1261 │   │   │   result = cast(LocallyExecutable, entity).local_execute(child_ctx, **kwargs)                  │
│                                                                                                                 │
│ /Users/tchase/anaconda3/envs/union-poc-env/lib/python3.11/site-packages/flytekit/core/base_task.py:335 in       │
│ local_execute                                                                                     

In [ ]:
# Test split_data task

X_train, X_val, X_test, y_train, y_val, y_test = split_data(data=data)
print(len(X_train))
print(len(X_val))
print(len(X_test))

In [ ]:
# Test create_search_grid task

hyperparameters = create_search_grid(grid)
hyperparameters

In [ ]:
# Test train_model task in same way as used in the workflow

partial_function = partial(train_model, X_train=X_train, y_train=y_train)
map_task_obj = map_task(partial_function)
models = map_task_obj(hyperparameters=hyperparameters)

In [ ]:
models[0].max_depth

In [ ]:
getattr(models[0], 'max_depth')

In [ ]:
# Test compare_model_results task

best_model = compare_model_results(X_val, y_val, models, hyperparameters, force_plot=True)
best_model

In [ ]:
analyze_model(
    X_train,
    y_train,
    X_val,
    y_val,
    X_test,
    y_test,
    best_model,
    force_plot=True
)

## Test Flyte Workflow

In [ ]:
# Test training_workflow workflow

best_model = training_workflow()
best_model